In [1]:
# script to calculate ratios for use in TTS method 
# match rf 11 to varying BL for ratios 
# 1) RF11 UT to RF11 BL 
# 2) RF11 UT to campaign average BL 
# 3) RF11 UT to RF11, 10, 09 BL 
# author: sofia chelpon
# date created: 3-28-2020
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns
import datetime as dt
from scipy.optimize import least_squares

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [3]:
# load in both toga and awas data pickles 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_replace_twp.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df_twp.pkl")
#toga_df.drop('WIC', axis = 1)

In [4]:
# count number of samples per RF 
utsamples = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)]
utsamples.groupby('Flight').count()

,Time_UTC,Notes,Intrument,GGALT,GGLAT,GGLON,Acetaldehyde,n_Butane,i_Butane,Benzene,C3H8,Methanol,Bromoform,Tetrachloroethylene,Dibromomethane,Dichloromethane,Chloroform,Carbon_Tetrachloride
Flight,,,,,,,,,,,,,,,,,,
RF03,39,39,39,39,39,39,22,39,39,39,39,39,39,39,39,39,39,39
RF04,90,90,90,90,90,90,74,87,87,87,87,86,87,87,87,87,87,87
RF05,96,96,96,96,96,96,66,95,95,95,95,95,95,95,95,95,95,95
RF06,39,39,39,39,39,39,28,39,39,39,39,37,39,39,39,39,39,39
RF07,44,44,44,44,44,44,33,43,43,43,43,43,43,43,43,43,43,43
RF08,48,48,48,48,48,48,37,48,48,48,48,48,48,48,48,47,48,48
RF09,48,48,48,48,48,48,34,48,48,48,48,45,48,48,48,48,48,48
RF10,75,75,75,75,75,75,59,65,65,65,65,65,65,65,65,65,65,65
RF11,54,54,54,54,54,54,29,52,52,52,52,52,52,52,52,52,52,52


In [17]:
utsamples = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)]
utsamples.groupby('Flight').count()

,Time_UTC,Notes,Intrument,GGALT,GGLAT,GGLON,C2H6_Ethane,C2H2_Ethyne,C3H8_Propane,C4H10_Isobutane,...,C2Br2F4_Halon_2402,CH2BrCl_Bromochloromethane,CH2Br2_Methylene_bromide,CHBrCl2_Bromodichloromethane,CHBr2Cl_Dibromochloromethane,CHBr3_Bromoform,C2H5ONO2_Ethyl_nitrate,C3H7NO3_Isopropyl_nitrate,C4H9NO3_2_Butyl_nitrate,C4H9NO3_n_Butyl_nitrate
Flight,,,,,,,,,,,,,,,,,,,,,
RF05,16,16,16,16,16,16,16,16,16,16,...,16,16,16,10,16,16,16,16,16,16
RF06,11,11,11,11,11,11,11,11,11,11,...,11,10,11,11,11,11,11,11,11,11
RF07,13,13,13,13,13,13,13,13,13,13,...,12,13,13,12,12,13,13,13,13,13
RF08,12,12,12,12,12,12,12,12,12,12,...,12,10,12,12,12,12,12,12,12,12
RF09,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
RF10,13,13,13,13,13,13,13,13,13,13,...,12,12,13,9,13,13,13,13,13,13
RF11,12,12,12,12,12,12,12,12,12,12,...,12,12,12,9,12,12,12,12,12,12
RF12,27,27,27,27,27,27,27,27,27,27,...,25,25,27,27,27,27,27,27,27,27
RF13,40,40,40,40,40,40,40,40,40,40,...,39,39,40,33,40,40,40,40,40,40


In [5]:
################# -------------------- TOGA  -------------------- #################

In [6]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
toga_means = pd.DataFrame()

# each rf 
#toga_means['UT - RF03'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
#                                      & (toga_df['Flight'] == 'RF03')].mean()
#toga_means['UT - RF04'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
#                                      & (toga_df['Flight'] == 'RF04')].mean()
toga_means['UT - RF05'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF05')].mean()
toga_means['UT - RF06'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF06')].mean()
toga_means['UT - RF07'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF07')].mean()
toga_means['UT - RF08'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF08')].mean()
toga_means['UT - RF09'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF09')].mean()
toga_means['UT - RF10'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF10')].mean()
toga_means['UT - RF11'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF11')].mean()
toga_means['UT - RF12'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF12')].mean()
toga_means['UT - RF13'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF13')].mean()
toga_means['UT - RF14'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF14')].mean()
toga_means['UT - RF15'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                      & (toga_df['Flight'] == 'RF15')].mean()
# all rf 
toga_means['BL - All RF'] = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()

toga_means = toga_means.drop('GGALT').drop('GGLAT').drop('GGLON')
#toga_means.index.name = "Trace_Gas"
toga_means = toga_means.reset_index()
toga_means = toga_means.rename(columns={'index': 'Trace_Gas'})
toga_means

,Trace_Gas,UT - RF05,UT - RF06,UT - RF07,UT - RF08,UT - RF09,UT - RF10,UT - RF11,UT - RF12,UT - RF13,UT - RF14,UT - RF15,BL - All RF
0,Acetaldehyde,3.075758,6.589286,35.454544,37.454052,38.841179,2.957627,63.565517,38.033333,11.053572,17.228125,11.133334,193.369537
1,n_Butane,3.251895,9.955896,6.179535,2.405000,2.749375,4.172153,2.065577,0.582093,3.701280,1.433182,3.640000,16.862217
2,i_Butane,2.506632,7.221539,5.402326,2.281875,2.770833,3.405539,2.226346,0.656977,3.147360,1.543182,3.318333,11.421493
3,Benzene,6.386842,12.191794,15.953953,7.870208,11.163124,11.448153,12.662115,5.625116,8.973920,7.825455,8.785000,24.730907
4,C3H8,33.115791,79.153847,66.697678,33.291668,32.229168,41.738461,33.788460,22.732557,48.208000,24.590910,41.500000,91.162895
5,Methanol,149.757889,80.270271,263.441864,228.541672,299.933319,198.107697,332.134613,293.058136,268.488007,284.681824,314.000000,637.142212
6,Bromoform,0.437095,0.357513,0.758721,0.771542,0.900104,0.737169,0.616135,0.740442,0.673000,0.459341,0.484000,0.954724
7,Tetrachloroethylene,0.850421,0.861282,1.248837,1.045208,1.151250,0.953385,1.100385,0.926512,1.022320,0.901364,0.931667,1.558500
8,Dibromomethane,0.974095,1.037154,1.081628,1.058250,1.166750,1.096369,1.005308,1.036477,1.026776,0.942159,0.731667,1.065606
9,Dichloromethane,40.344425,50.733074,50.959301,42.560001,42.993958,45.204155,49.803082,41.042091,42.894882,40.147274,44.493336,57.213898


In [7]:
# ratios for 1) flights in total and 2) individual flights 
toga_ratios = pd.DataFrame()

toga_ratios['Trace_Gas'] = toga_means['Trace_Gas']

#toga_ratios['RF03_CampAvg'] = toga_means['UT - RF03']/toga_means['BL - All RF']
#toga_ratios['RF04_CampAvg'] = toga_means['UT - RF04']/toga_means['BL - All RF']
toga_ratios['RF05_CampAvg'] = toga_means['UT - RF05']/toga_means['BL - All RF']
toga_ratios['RF06_CampAvg'] = toga_means['UT - RF06']/toga_means['BL - All RF']
toga_ratios['RF07_CampAvg'] = toga_means['UT - RF07']/toga_means['BL - All RF']
toga_ratios['RF08_CampAvg'] = toga_means['UT - RF08']/toga_means['BL - All RF']
toga_ratios['RF09_CampAvg'] = toga_means['UT - RF09']/toga_means['BL - All RF']
toga_ratios['RF10_CampAvg'] = toga_means['UT - RF10']/toga_means['BL - All RF']
toga_ratios['RF11_CampAvg'] = toga_means['UT - RF11']/toga_means['BL - All RF']
toga_ratios['RF12_CampAvg'] = toga_means['UT - RF12']/toga_means['BL - All RF']
toga_ratios['RF13_CampAvg'] = toga_means['UT - RF13']/toga_means['BL - All RF']
toga_ratios['RF14_CampAvg'] = toga_means['UT - RF14']/toga_means['BL - All RF']
toga_ratios['RF15_CampAvg'] = toga_means['UT - RF15']/toga_means['BL - All RF']

toga_ratios

# drop species names so doesnt duplicate column when merged 
toga_ratios = toga_ratios.drop('Trace_Gas', axis = 1)

In [8]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_08042020.xlsx'
toga_lifetimes = pd.read_excel(mypath2)

# merge lifetimes to ratios
toga_ratios_full = toga_lifetimes.merge(toga_ratios, left_index=True, right_index=True)
toga_ratios_full.insert(0, 'Instrument', 'TOGA')

In [9]:
toga_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,RF05_CampAvg,RF06_CampAvg,RF07_CampAvg,RF08_CampAvg,RF09_CampAvg,RF10_CampAvg,RF11_CampAvg,RF12_CampAvg,RF13_CampAvg,RF14_CampAvg,RF15_CampAvg
0,TOGA,Acetaldehyde,0.283684,0.321694,0.348720,0.015906,0.034076,0.183351,0.193692,0.200865,0.015295,0.328726,0.196687,0.057163,0.089094,0.057575
1,TOGA,n_Butane,1.979710,3.901968,8.096203,0.192851,0.590426,0.366472,0.142627,0.163049,0.247426,0.122497,0.034521,0.219501,0.084994,0.215867
2,TOGA,i_Butane,2.193483,4.101641,7.916587,0.219466,0.632276,0.472996,0.199788,0.242598,0.298169,0.194926,0.057521,0.275565,0.135112,0.290534
3,TOGA,Benzene,3.819659,6.602596,12.137543,0.258253,0.492978,0.645102,0.318234,0.451384,0.462909,0.511996,0.227453,0.362863,0.316424,0.355224
4,TOGA,C3H8,4.276704,9.379963,22.541651,0.363260,0.868268,0.731632,0.365189,0.353534,0.457845,0.370638,0.249362,0.528812,0.269747,0.455229
5,TOGA,Methanol,5.198347,9.757627,20.088572,0.235046,0.125985,0.413474,0.358698,0.470748,0.310932,0.521288,0.459957,0.421394,0.446810,0.492826
6,TOGA,Bromoform,12.898712,18.666902,24.882113,0.457823,0.374467,0.794702,0.808131,0.942790,0.772128,0.645354,0.775556,0.704916,0.481124,0.506953
7,TOGA,Tetrachloroethylene,28.154334,83.261722,320.000000,0.545666,0.552635,0.801307,0.670650,0.738691,0.611732,0.706054,0.594489,0.655964,0.578353,0.597797
8,TOGA,Dibromomethane,39.940949,100.000000,310.000000,0.914123,0.973300,1.015036,0.993097,1.094917,1.028870,0.943414,0.972665,0.963561,0.884154,0.686621
9,TOGA,Dichloromethane,46.722142,120.000000,350.000000,0.705151,0.886726,0.890680,0.743875,0.751460,0.790090,0.870472,0.717345,0.749728,0.701705,0.777667


In [10]:
################# -------------------- AWAS  -------------------- #################

In [11]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
awas_means = pd.DataFrame()

# each rf
#awas_means['UT - RF03'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
#                                      & (awas_df['Flight'] == 'RF03')].mean()
#awas_means['UT - RF04'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
#                                      & (awas_df['Flight'] == 'RF04')].mean()
awas_means['UT - RF05'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF05')].mean()
awas_means['UT - RF06'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF06')].mean()
awas_means['UT - RF07'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF07')].mean()
awas_means['UT - RF08'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF08')].mean()
awas_means['UT - RF09'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF09')].mean()
awas_means['UT - RF10'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF10')].mean()
awas_means['UT - RF11'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF11')].mean()
awas_means['UT - RF12'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF12')].mean()
awas_means['UT - RF13'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF13')].mean()
awas_means['UT - RF14'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF14')].mean()
awas_means['UT - RF15'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                      & (awas_df['Flight'] == 'RF15')].mean()

# all rf 
awas_means['BL - All RF'] = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()

awas_means = awas_means.drop('GGALT').drop('GGLAT').drop('GGLON')
#toga_means.index.name = "Trace_Gas"
awas_means = awas_means.reset_index()
awas_means = awas_means.rename(columns={'index': 'Trace_Gas'})
awas_means

,Trace_Gas,UT - RF05,UT - RF06,UT - RF07,UT - RF08,UT - RF09,UT - RF10,UT - RF11,UT - RF12,UT - RF13,UT - RF14,UT - RF15,BL - All RF
0,C2H6_Ethane,446.500000,621.636353,533.461548,442.916656,494.111115,461.076935,504.916656,435.037048,521.349976,473.777771,516.799988,821.637939
1,C2H2_Ethyne,45.312500,92.545456,78.846153,47.000000,63.333332,66.307693,71.416664,37.444443,52.700001,47.222221,57.799999,121.142860
2,C3H8_Propane,34.750000,84.727272,58.769230,33.083332,39.555557,42.461540,34.583332,20.444445,45.599998,26.888889,36.000000,143.758621
3,C4H10_Isobutane,2.812500,9.536364,5.353846,2.300000,2.477778,3.507693,2.066667,1.029630,2.697500,1.300000,1.740000,15.060345
4,C4H10_nButane,5.243750,18.809092,8.638461,3.325000,3.411111,4.684615,3.466666,1.929630,4.330000,1.988889,3.220000,24.112068
5,C5H12_nPentane,1.100000,2.309091,1.592308,0.354545,0.442857,0.737500,0.310000,0.400000,0.273684,0.150000,0.366667,4.175510
6,C6H6_Benzene,6.599999,16.545454,16.161541,6.625000,10.633333,12.184616,14.908334,6.162963,8.730000,7.100000,8.580000,26.858931
7,OCS_Carbonyl_sulfide,557.687500,560.727295,561.076904,556.250000,549.111084,552.769226,548.083313,551.846130,537.150024,542.111084,535.799988,546.824585
8,C2H6S_Dimethyl_sulfide,0.585000,0.186667,0.348000,0.146667,0.250000,1.320000,0.124000,0.182857,0.178261,0.140000,0.080000,16.533684
9,CCl2F2_CFC_12,516.062500,520.454529,518.818176,524.090881,511.142853,516.250000,520.416687,519.640015,518.606079,523.111084,519.799988,517.571411


In [12]:
# ratios for 1) flights in total and 2) individual flights 
awas_ratios = pd.DataFrame()

#awas_ratios['RF03_CampAvg'] = awas_means['UT - RF03']/awas_means['BL - All RF']
#awas_ratios['RF04_CampAvg'] = awas_means['UT - RF04']/awas_means['BL - All RF']
awas_ratios['RF05_CampAvg'] = awas_means['UT - RF05']/awas_means['BL - All RF']
awas_ratios['RF06_CampAvg'] = awas_means['UT - RF06']/awas_means['BL - All RF']
awas_ratios['RF07_CampAvg'] = awas_means['UT - RF07']/awas_means['BL - All RF']
awas_ratios['RF08_CampAvg'] = awas_means['UT - RF08']/awas_means['BL - All RF']
awas_ratios['RF09_CampAvg'] = awas_means['UT - RF09']/awas_means['BL - All RF']
awas_ratios['RF10_CampAvg'] = awas_means['UT - RF10']/awas_means['BL - All RF']
awas_ratios['RF11_CampAvg'] = awas_means['UT - RF11']/awas_means['BL - All RF']
awas_ratios['RF12_CampAvg'] = awas_means['UT - RF12']/awas_means['BL - All RF']
awas_ratios['RF13_CampAvg'] = awas_means['UT - RF13']/awas_means['BL - All RF']
awas_ratios['RF14_CampAvg'] = awas_means['UT - RF14']/awas_means['BL - All RF']
awas_ratios['RF15_CampAvg'] = awas_means['UT - RF15']/awas_means['BL - All RF']

# drop species names so doesnt duplicate column when merged 
#awas_ratios = awas_ratios.drop('Trace_Gas', axis = 1)
awas_ratios

,RF05_CampAvg,RF06_CampAvg,RF07_CampAvg,RF08_CampAvg,RF09_CampAvg,RF10_CampAvg,RF11_CampAvg,RF12_CampAvg,RF13_CampAvg,RF14_CampAvg,RF15_CampAvg
0,0.543427,0.756582,0.649266,0.539065,0.601373,0.561168,0.614525,0.529475,0.634525,0.576626,0.628987
1,0.374042,0.763937,0.650853,0.387972,0.522799,0.547351,0.589524,0.309093,0.435024,0.389806,0.477123
2,0.241725,0.589372,0.408805,0.230131,0.275153,0.295367,0.240565,0.142214,0.317198,0.187042,0.250420
3,0.186749,0.633210,0.355493,0.152719,0.164523,0.232909,0.137226,0.068367,0.179113,0.086319,0.115535
4,0.217474,0.780070,0.358263,0.137898,0.141469,0.194285,0.143773,0.080028,0.179578,0.082485,0.133543
5,0.263441,0.553008,0.381344,0.084911,0.106061,0.176625,0.074242,0.095797,0.065545,0.035924,0.087814
6,0.245728,0.616013,0.601719,0.246659,0.395896,0.453652,0.555061,0.229457,0.325032,0.264344,0.319447
7,1.019865,1.025424,1.026064,1.017237,1.004181,1.010871,1.002302,1.009183,0.982308,0.991380,0.979839
8,0.035382,0.011290,0.021048,0.008871,0.015121,0.079837,0.007500,0.011060,0.010782,0.008468,0.004839
9,0.997085,1.005570,1.002409,1.012596,0.987579,0.997447,1.005497,1.003997,1.001999,1.010703,1.004306


In [13]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_08042020.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
len(awas_lifetimes)

# merge lifetimes to ratios 
awas_ratios_full = awas_lifetimes.merge(awas_ratios, left_index=True, right_index=True)
awas_ratios_full.insert(0, 'Instrument', 'AWAS')

In [14]:
################# -------------------- MERGE THE TWO  -------------------- #################

In [15]:
master_list = awas_ratios_full
master_list = master_list.append(toga_ratios_full, ignore_index = True, sort = False)

# pickle this!! 
master_list.to_pickle("./contrast_ratios_perrf.pkl")

# look at output 
#master_list[master_list['Trace_Gas'] == 'Dichloromethane'].drop('BL_tau', axis = 1).drop('UT_tau', axis = 1)

In [16]:
master_list

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,RF05_CampAvg,RF06_CampAvg,RF07_CampAvg,RF08_CampAvg,RF09_CampAvg,RF10_CampAvg,RF11_CampAvg,RF12_CampAvg,RF13_CampAvg,RF14_CampAvg,RF15_CampAvg
0,AWAS,C2H6_Ethane,18.996560,51.867565,170.000000,0.543427,0.756582,0.649266,0.539065,0.601373,0.561168,0.614525,0.529475,0.634525,0.576626,0.628987
1,AWAS,C2H2_Ethyne,5.460657,9.761061,18.790019,0.374042,0.763937,0.650853,0.387972,0.522799,0.547351,0.589524,0.309093,0.435024,0.389806,0.477123
2,AWAS,C3H8_Propane,4.276704,9.379963,22.541651,0.241725,0.589372,0.408805,0.230131,0.275153,0.295367,0.240565,0.142214,0.317198,0.187042,0.250420
3,AWAS,C4H10_Isobutane,2.193483,4.101641,7.916587,0.186749,0.633210,0.355493,0.152719,0.164523,0.232909,0.137226,0.068367,0.179113,0.086319,0.115535
4,AWAS,C4H10_nButane,1.979710,3.901968,8.096203,0.217474,0.780070,0.358263,0.137898,0.141469,0.194285,0.143773,0.080028,0.179578,0.082485,0.133543
5,AWAS,C5H12_nPentane,1.225894,2.360346,4.742530,0.263441,0.553008,0.381344,0.084911,0.106061,0.176625,0.074242,0.095797,0.065545,0.035924,0.087814
6,AWAS,C6H6_Benzene,3.819659,6.602596,12.137543,0.245728,0.616013,0.601719,0.246659,0.395896,0.453652,0.555061,0.229457,0.325032,0.264344,0.319447
7,AWAS,OCS_Carbonyl_sulfide,2000.000000,2000.000000,2000.000000,1.019865,1.025424,1.026064,1.017237,1.004181,1.010871,1.002302,1.009183,0.982308,0.991380,0.979839
8,AWAS,C2H6S_Dimethyl_sulfide,0.655356,0.377066,0.407900,0.035382,0.011290,0.021048,0.008871,0.015121,0.079837,0.007500,0.011060,0.010782,0.008468,0.004839
9,AWAS,CCl2F2_CFC_12,37000.000000,37000.000000,37000.000000,0.997085,1.005570,1.002409,1.012596,0.987579,0.997447,1.005497,1.003997,1.001999,1.010703,1.004306
